### Data Collection





In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
import os
import json
from requests_oauthlib import OAuth1Session
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#from newsapi import NewsApiClient
from pathlib import Path
import tweepy as tw
%matplotlib inline
load_dotenv()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jakek\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [35]:
df_1 = pd.read_csv('tweets1.csv', usecols = ['id', 'created_at', 'text'])
df_2 = pd.read_csv('tweets2.csv', usecols = ['id', 'created_at', 'text'])
df_3 = pd.read_csv('tweets3.csv', usecols = ['id', 'created_at', 'text'])
df_4 = pd.read_csv('tweets4.csv', usecols = ['id', 'created_at', 'text'])

In [ ]:
df_total = pd.concat([df_1,df_2,df_3,df_4])
df_total[df_total['$'].replace("DOLLARS")]

In [61]:
df_scrub = df_total[df_total['text'].str.contains("$", regex=False)]
df_scrub

,id,created_at,text
63,1.500000e+18,2022-03-11T18:17:03.000Z,*83 COUNTERPARTIES TAKE $1.558 TLN AT FED REVE...
64,1.500000e+18,2022-03-11T18:15:27.000Z,*SEC AWARDS $14M TO WHISTLEBLOWER WHO EXPOSED ...
3,1.500000e+18,2022-03-12T03:04:33.000Z,"@RedRockTrader1 Yes, I missed the short after ..."
5,1.500000e+18,2022-03-12T00:33:56.000Z,Sometimes you just have to ask for the VWAP sq...
11,1.500000e+18,2022-03-11T22:54:37.000Z,Holy $DIDI just saw it ????
...,...,...,...
78,1.500000e+18,2022-03-09T20:25:04.000Z,$CCJ Notable Options Flow in Open Interest (So...
79,1.500000e+18,2022-03-09T20:19:43.000Z,Getting it! $CCJ https://t.co/NiqQcA0bAW
83,1.500000e+18,2022-03-09T18:30:30.000Z,"$SPY\r\n\r\nWhy 414/415 was so important, last..."
87,1.500000e+18,2022-03-09T17:14:08.000Z,"$ABBV highs, tight bull flag forming for what ..."


In [62]:
tw_sentiments = []

for text in df_scrub["text"]:
    try:

        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        tw_sentiments.append({
            "text": text,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })

    except AttributeError:
        pass

In [63]:
tw_df = pd.DataFrame(tw_sentiments)

In [64]:
cols = ["text", "compound", "positive", "negative", "neutral"]
tw_df = tw_df[cols]
tw_df = tw_df.sort_values('compound', ascending = False)
tw_df

,text,compound,positive,negative,neutral
53,$ISPO I liked it last time when it went straig...,0.8760,0.459,0.000,0.541
15,$HYMC .50/share ago and 280 million shares tra...,0.8426,0.238,0.000,0.762
86,$CMG notable early strength above its 8-MA\r\n...,0.8126,0.261,0.000,0.739
31,$GFAI just so you know SSR is on -- so they ha...,0.7506,0.144,0.039,0.817
49,I don't know who did it better -- The $DATS de...,0.7269,0.210,0.000,0.790
...,...,...,...,...,...
87,$NVDA weak down 3.5% under yesterday low after...,-0.6124,0.000,0.294,0.706
56,One of those days you're happy $AGRI borrow wa...,-0.6249,0.152,0.221,0.626
85,$OLLI large Apr. put spread bought\r\n\r\nLate...,-0.8055,0.000,0.248,0.752
25,$GFAI speaking of the devil ????\r\n\r\nNice w...,-0.8205,0.084,0.312,0.603


In [65]:
# Top 5 tickers with highest compound sentiment analysis 
## ISPO, HYMC, CMG, GFAI and DATS

,compound,positive,negative,neutral
count,99.000000,99.000000,99.000000,99.000000
mean,0.134682,0.110717,0.057061,0.832222
std,0.380574,0.142832,0.107167,0.182052
min,-0.892200,0.000000,0.000000,0.196000
25%,0.000000,0.000000,0.000000,0.734500
50%,0.000000,0.071000,0.000000,0.864000
75%,0.424600,0.176000,0.053000,1.000000
max,0.876000,0.625000,0.423000,1.000000
